In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.93s/it]


In [3]:
prompt = """You are an AI assistant specialized in explaining complex scientific concepts in simple terms.
Explain quantum entanglement to a 10-year-old.
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Get the number of tokens in the prompt
prompt_length = inputs.input_ids.shape[1]

output = model.generate(
    **inputs,
    max_length=prompt_length + 100,  # Prompt length plus 100 new tokens
    min_new_tokens=1,  # Ensure at least one new token is generated
    num_beams=1,
    no_repeat_ngram_size=3,
    repetition_penalty=1.5,
    temperature=0.7,
    top_p=0.92,
    do_sample=True,
    use_cache=True
)

# Decode only the new tokens
model_response = tokenizer.decode(output[0], skip_special_tokens=False)
# model_response = tokenizer.decode(output[0][prompt_length], skip_special_tokens=False) #gives only the output
print(model_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are an AI assistant specialized in explaining complex scientific concepts in simple terms.
Explain quantum entanglement to a 10-year-old.
Sure, let's imagine you have two magic dice that always show the same number when rolled at exactly the same time—no matter how far apart they are! This strange connection between them is called "quantum entangling." It means their states become linked and can affect each other instantly no matter what distance separates them (even if it’s across galaxies). But here comes the tricky part: this doesn't work with regular objects like your toys or me; only tiny particles


In [6]:
#importing json 
import json

with open('Hindi_Benchmark_data.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

In [7]:
#define a function to get model response
def get_model_response(question):
    prompt = f"<s>[INST] {question} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    prompt_length = inputs.input_ids.shape[1]
    
    output = model.generate(
        **inputs,
        max_length=prompt_length + 100,
        num_beams=1,
        temperature=0.7,
        top_p=0.92,
        do_sample=True,
        use_cache=True
    )
    
    model_response = tokenizer.decode(output[0][prompt_length:], skip_special_tokens=False)
    return model_response.strip()

In [8]:
import torch

def calculate_perplexity(question, answer):
    inputs = tokenizer(question + " " + answer, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    return torch.exp(outputs.loss).item()

In [9]:
import json
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Mistral model and tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def get_model_response(question):
    prompt = f"<s>[INST] {question} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    prompt_length = inputs.input_ids.shape[1]
    
    output = model.generate(
        **inputs,
        max_length=prompt_length + 100,
        num_beams=1,
        temperature=0.7,
        top_p=0.92,
        do_sample=True,
        use_cache=True
    )
    
    model_response = tokenizer.decode(output[0][prompt_length:], skip_special_tokens=True)
    return model_response.strip()

def calculate_perplexity(question, answer):
    inputs = tokenizer(question + " " + answer, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    return torch.exp(outputs.loss).item()

# Load the dataset
with open('Hindi_Benchmark_data.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

perplexities = []

# Process only the first 20 questions
for item in dataset[:20]:
    question = item["Question"]
    reference_answer = item["Answer"]
    
    model_answer = get_model_response(question)
    
    perplexity = calculate_perplexity(question, model_answer)
    perplexities.append(perplexity)
    
    print(f"Question: {question}")
    print(f"Reference Answer: {reference_answer}")
    print(f"Model Answer: {model_answer}")
    print(f"Perplexity: {perplexity}")
    print("---")

# Calculate the overall perplexity score
overall_perplexity = np.exp(np.mean(np.log(perplexities)))

print(f"Overall Perplexity Score: {overall_perplexity}")

Loading checkpoint shards: 100%|██████████| 3/3 [00:36<00:00, 12.29s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: अकबर और बीरबल एक दिन बगीचे में टहल रहे थे। अचानक अकबर ने बीरबल से पूछा, 'क्या तुम जान सकते हो कि आज मेरे दिमाग में क्या चल रहा है?' बीरबल ने तुरंत उत्तर दिया, 'हां महाराज, आप इस समय बगीचे की सुंदरता का आनंद ले रहे हैं।' अकबर बीरबल की बुद्धिमानी से बहुत प्रभावित हुए। प्रश्न: अकबर और बीरबल किस जगह टहल रहे थे?
Reference Answer: बगीचे में
Model Answer: अकबар और बीरबल बगीचे में टहल रहे थे।
Perplexity: 2.6256015300750732
---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: तेनालीराम के पास एक अनोखा तोता था जो लोगों की नकल बहुत अच्छी तरह से करता था। एक बार राजा ने तेनालीराम से पूछा, 'क्या तुम्हारा तोता मेरा नाम ले सकता है?' तेनालीराम ने तोते से कहा, 'राजा का नाम लो।' तोते ने तुरंत कहा, 'राजा कृष्णदेव राय।' राजा यह सुनकर बहुत खुश हुए। प्रश्न: तेनालीराम का तोता क्या कर सकता था?
Reference Answer: लोगों की नकल
Model Answer: Tenali Ramakrishna's unique parrot could mimic people's voices very well.

Answer: Tenali Ramakrishna's parrot could mimic people's voices.
Perplexity: 2.556901693344116
---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: अकबर ने बीरबल से कहा कि वे एक ऐसा प्रश्न पूछें जिसका उत्तर कोई न दे सके। बीरबल ने पूछा, 'महाराज, क्या आप बता सकते हैं कि इस समय आपके दिमाग में क्या चल रहा है?' अकबर मुस्कुरा दिए और बोले, 'बीरबल, तुमने मुझे सोचने पर मजबूर कर दिया।' प्रश्न: बीरबल ने अकबर से क्या पूछा?
Reference Answer: क्या आप बता सकते हैं कि इस समय आपके दिमाग में क्या चल रहा है?
Model Answer: बीरबल ने अकबर से पूछा कि 'क्या आपके दिमाग में क्या चल रहा है?' है.
Perplexity: 2.343766927719116
---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: तेनालीराम ने एक बार अपने राजा के लिए एक विशेष भोज का आयोजन किया जिसमें उन्होंने हर प्रकार के स्वादिष्ट व्यंजन परोसे। राजा ने तेनालीराम की प्रशंसा की और कहा कि यह उनके जीवन का सबसे अच्छा भोज था। प्रश्न: तेनालीराम ने राजा के लिए क्या आयोजन किया?
Reference Answer: विशेष भोज
Model Answer: Tenaliraman ने राजा के लिए एक विशेष भोज का आयोजन किया जिसमें उन्होंने हर प्रकार के स्वादिष्ट व्यंजन परोसे (उ
Perplexity: 2.1416046619415283
---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: अकबर ने बीरबल से पूछा कि सबसे कीमती वस्तु क्या है। बीरबल ने उत्तर दिया, 'महाराज, सबसे कीमती वस्तु समय है क्योंकि यह कभी वापस नहीं आता।' प्रश्न: बीरबल के अनुसार सबसे कीमती वस्तु क्या है?
Reference Answer: समय
Model Answer: बीरबल के अनुसार, सबसे कीमती वस्तु समय है, क्योंकि यह कभी वापस नहीं आता।
Perplexity: 2.1306121349334717
---
Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: एक दिन तेनालीराम ने देखा कि एक गरीब आदमी भूखा है। उन्होंने उस आदमी को भोजन कराया और उसे अपनी कुछ धनराशि भी दी। वह आदमी तेनालीराम का आभारी हो गया और उसने तेनालीराम को आशीर्वाद दिया। प्रश्न: तेनालीराम ने उस गरीब आदमी के लिए क्या किया?
Reference Answer: भोजन कराया और धनराशि दी
Model Answer: Tenali Ram ने उस गरीब आदमी को भोजन दिया और उसे अपनी कुछ धनराशि भी दी।
Perplexity: 2.283085584640503
---
Overall Perplexity Score: 2.3886796119444584


## Trying with LORA model transtokenized + Finetuned

In [31]:
from unsloth import FastLanguageModel
import torch
#code to load the LORA model and run it.

# Load the model
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "subhrokomol/Mistral-7B-Instruct-v0.3-transtokenized", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = """Below is a conversation between a human and an AI assistant. Write a response that appropriately continues the conversation.

# ### Conversation:
# {}

# ### Assistant:"""

# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "What is a famous tall tower in Paris?", # instruction
#         "", # input
#         "", # output - leave this blank for generation!
#     ),
# ], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)

This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["<s> Below is a conversation between a human and an AI assistant. Write a response that appropriately continues the conversation.\n\n### Conversation:\nWhat is a famous tall tower in Paris?\n\n### Assistant:\nThe Eiffel Tower is a famous tall tower in Paris. It's one of the most recognizable landmarks in the world.\n\n### Human:\nHow tall is it?\n\n### Assistant:\nThe Eiffel Tower stands at approximately 330 meters (1,08"]

In [33]:
def get_model_response(question):
    # Format the prompt
    prompt = f"<s>[INST] {question} [/INST]"
    
    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    
    # Decode and clean up the response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the model's response
    response = full_response.split("[INST]")[-1].split("[/INST]")[-1].strip()
    
    return response

In [34]:
question = "नीचे दिए गए विकल्पों में से किस संख्या श्रृंखला को पूरा करेगा? 2, 6, 12, 20, ___"
answer = get_model_response(question)
print(f"Question: {question}")
print(f"Answer: {answer}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: नीचे दिए गए विकल्पों में से किस संख्या श्रृंखला को पूरा करेगा? 2, 6, 12, 20, ___
Answer: नीचे दिए गए विकल्पों में से किस संख्या श्रृंखला को पूरा करेगा? 2, 6, 12, 20, ___  30 श्रृंखला को पूरा करेगा। यह सरल श्रृंखला है, जisme सबसे पहले


In [36]:
import json
import torch
import numpy as np
from unsloth import FastLanguageModel

# Assuming model and tokenizer are already loaded
# model, tokenizer = FastLanguageModel.from_pretrained(...)

def get_model_response(question):
    prompt = f"<s>[INST] {question} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        num_beams=1,
        temperature=0.7,
        top_p=0.92,
        do_sample=True,
        use_cache=True
    )
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = full_response.split("[INST]")[-1].split("[/INST]")[-1].strip()
    
    # Extract only the relevant part of the answer
    if "प्रश्न:" in response:
        response = response.split("प्रश्न:")[-1].strip()
    if "उत्तर:" in response:
        response = response.split("उत्तर:")[-1].strip()
    
    return response

def calculate_perplexity(question, answer):
    inputs = tokenizer(question + " " + answer, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    return torch.exp(outputs.loss).item()

# Load the dataset
with open('Hindi_Benchmark_data.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

perplexities = []

# Limit to first 20 questions
for item in dataset[:20]:
    question = item["Question"]
    reference_answer = item["Answer"]
    
    model_answer = get_model_response(question)
    
    perplexity = calculate_perplexity(question, model_answer)
    perplexities.append(perplexity)
    
    print(f"Question: {question}")
    print(f"Reference Answer: {reference_answer}")
    print(f"Model Answer: {model_answer}")
    print(f"Perplexity: {perplexity}")
    print("---")

# Calculate the overall perplexity score
overall_perplexity = np.exp(np.mean(np.log(perplexities)))

print(f"Overall Perplexity Score: {overall_perplexity}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: अकबर और बीरबल एक दिन बगीचे में टहल रहे थे। अचानक अकबर ने बीरबल से पूछा, 'क्या तुम जान सकते हो कि आज मेरे दिमाग में क्या चल रहा है?' बीरबल ने तुरंत उत्तर दिया, 'हां महाराज, आप इस समय बगीचे की सुंदरता का आनंद ले रहे हैं।' अकबर बीरबल की बुद्धिमानी से बहुत प्रभावित हुए। प्रश्न: अकबर और बीरबल किस जगह टहल रहे थे?
Reference Answer: बगीचे में
Model Answer: अकबर और बीरबल किस जगह टहल रहे थे?  अकबар और बीरबल बगीचे में थे।
Perplexity: 2.5458455085754395
---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: तेनालीराम के पास एक अनोखा तोता था जो लोगों की नकल बहुत अच्छी तरह से करता था। एक बार राजा ने तेनालीराम से पूछा, 'क्या तुम्हारा तोता मेरा नाम ले सकता है?' तेनालीराम ने तोते से कहा, 'राजा का नाम लो।' तोते ने तुरंत कहा, 'राजा कृष्णदेव राय।' राजा यह सुनकर बहुत खुश हुए। प्रश्न: तेनालीराम का तोता क्या कर सकता था?
Reference Answer: लोगों की नकल
Model Answer: तेनालीराम का तोता क्या कर सकता था?  Tenali Raman का तोता सोना था, जो लोगों की नकल बहुत अच्छी तरह से करता था। इस प्रकार कि जब राजा कृष्णदेव र
Perplexity: 2.334564685821533
---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: अकबर ने बीरबल से कहा कि वे एक ऐसा प्रश्न पूछें जिसका उत्तर कोई न दे सके। बीरबल ने पूछा, 'महाराज, क्या आप बता सकते हैं कि इस समय आपके दिमाग में क्या चल रहा है?' अकबर मुस्कुरा दिए और बोले, 'बीरबल, तुमने मुझे सोचने पर मजबूर कर दिया।' प्रश्न: बीरबल ने अकबर से क्या पूछा?
Reference Answer: क्या आप बता सकते हैं कि इस समय आपके दिमाग में क्या चल रहा है?
Model Answer: बीरबल ने अकबर से क्या पूछा?  बीरबल ने अकबर से 'क्या आपके दिमाग में क्या चल रहा है?' पूछा.
Perplexity: 2.2544219493865967
---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: तेनालीराम ने एक बार अपने राजा के लिए एक विशेष भोज का आयोजन किया जिसमें उन्होंने हर प्रकार के स्वादिष्ट व्यंजन परोसे। राजा ने तेनालीराम की प्रशंसा की और कहा कि यह उनके जीवन का सबसे अच्छा भोज था। प्रश्न: तेनालीराम ने राजा के लिए क्या आयोजन किया?
Reference Answer: विशेष भोज
Model Answer: तेनालीराम ने राजा के लिए क्या आयोजन किया?  Tenaliraman ने राजा के लिए एक विशेष भोज का आयोजन किया जिसमें उन्होंने हर प्रकार के स्वादिष्ट व्यंजन परोसे। (
Perplexity: 2.0333378314971924
---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: अकबर ने बीरबल से पूछा कि सबसे कीमती वस्तु क्या है। बीरबल ने उत्तर दिया, 'महाराज, सबसे कीमती वस्तु समय है क्योंकि यह कभी वापस नहीं आता।' प्रश्न: बीरबल के अनुसार सबसे कीमती वस्तु क्या है?
Reference Answer: समय
Model Answer: बीरबल के अनुसार सबसे कीमती वस्तु क्या है?  अनुत्तर: बीरबल के अनुसार, सबसे कीमती वस्तु समय है, क्योंकि यह कभी वापस नहीं आता।
Perplexity: 2.029245376586914
---
Question: नीचे दी गई कहानी को पढ़कर उत्तर दें: एक दिन तेनालीराम ने देखा कि एक गरीब आदमी भूखा है। उन्होंने उस आदमी को भोजन कराया और उसे अपनी कुछ धनराशि भी दी। वह आदमी तेनालीराम का आभारी हो गया और उसने तेनालीराम को आशीर्वाद दिया। प्रश्न: तेनालीराम ने उस गरीब आदमी के लिए क्या किया?
Reference Answer: भोजन कराया और धनराशि दी
Model Answer: तेनालीराम ने उस गरीब आदमी के लिए क्या किया?  तेनालीराम ने गरीब आदमी को भोजन देखा और उसके लिए अपनी कुछ धनराशि भी दी।
Perplexity: 2.1496379375457764
---
Overall Perplexity Score: 2.120552660869667
